In [ ]:
import requests
import pandas as pd
import redshift_connector
from dotenv import dotenv_values
import psycopg2
import os
from io import StringIO
import sqlalchemy as sa
from sqlalchemy.engine.url import URL


    
env_vars = dotenv_values('.env')

In [ ]:
function = 'TIME_SERIES_WEEKLY'
# Big Five Tech: Google, Amazon, Meta, Apple, and Microsoft (GAMAM)
symbols = ['GOOG', 'AMZN', 'METV', 'AAPL', 'MSFT']
api_key = 'WNHSBPLUX5B8HNMZ'

df = pd.DataFrame()

for symbol in symbols:
    url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={api_key}'
    r = requests.get(url)
    data = r.json()

    symbol_df = pd.DataFrame(data['Weekly Time Series'])
    symbol_df = symbol_df.T
    symbol_df.reset_index(inplace=True)
    symbol_df.rename(columns={'index':'week_from', '1. open':'open', '2. high':'high', '3. low': 'low', 
                       '4. close':'close', '5. volume':'volume'}, inplace=True)
    
    symbol_df['symbol'] = data['Meta Data']['2. Symbol']

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['avg'] = (symbol_df['open'] + symbol_df['close'])/2
    symbol_df['pk'] = symbol_df['symbol']+symbol_df['week_from']

    # symbol_df['avg'] = symbol_df[['close', 'open']].mean(axis=1)
    
    df = pd.concat([df, symbol_df])

In [25]:
url = URL.create(
drivername='redshift+redshift_connector', # indicate redshift_connector driver and dialect will be used
host=env_vars['HOST'], # Amazon Redshift host
port=int(env_vars['PORT']), # Amazon Redshift port
database=env_vars['DATABASE'], # Amazon Redshift database
username=env_vars['USER'], # Amazon Redshift username
password=env_vars['PASSWORD'] # Amazon Redshift password
)

engine = sa.create_engine(url)


4289

In [28]:
# Connect to Redshift using psycopg2
conn = psycopg2.connect(
    host=env_vars['HOST'],
    port=int(env_vars['PORT']),
    database=env_vars['DATABASE'],
    user=env_vars['USER'],
    password=env_vars['PASSWORD']
 )

cursor = conn.cursor()

# Create table if not exists
tmp_table = '''CREATE TABLE IF NOT EXISTS laureanoengulian_coderhouse.stage(
                        "week_from" date not null,
                        "open" decimal(38, 4) not null,
                        "high" decimal(38, 4) not null,
                        "low" decimal(38, 4) not null,
                        "close" decimal(38, 4) not null,
                        "volume" bigint not null,
                        "symbol" varchar(15) not null,
                        "avg" decimal(38,4) not null,
                        "pk" varchar(60));
                     '''

cursor.execute(tmp_table)
conn.commit()

df.to_sql(name='stage',
          con=engine,
          if_exists='replace',
          index=False)
          # method=callable)

sql_transaction = ''' begin transaction;

                      delete from laureanoengulian_coderhouse.big_five_weekly using laureanoengulian_coderhouse.stage 
                      where big_five_weekly.pk = stage.pk;

                      insert into laureanoengulian_coderhouse.big_five_weekly
                      select * from laureanoengulian_coderhouse.stage;

                      end transaction;
                  '''

cursor.execute(sql_transaction)
conn.commit()

drop_tmp = '''drop table if exists laureanoengulian_coderhouse.stage;'''

cursor.execute(sql_transaction)
conn.commit()

DatatypeMismatch: column "volume" is of type bigint but expression is of type character varying
HINT:  You will need to rewrite or cast the expression.


In [ ]:
df.head()

In [30]:
# Connect to Redshift using psycopg2
conn = psycopg2.connect(
    host=env_vars['HOST'],
    port=int(env_vars['PORT']),
    database=env_vars['DATABASE'],
    user=env_vars['USER'],
    password=env_vars['PASSWORD']
 )

cursor = conn.cursor()

# # Eliminate the table if exists
# cleaner_query = '''DROP TABLE IF EXISTS laureanoengulian_coderhouse.big_five_weekly;'''
# cursor.execute(cleaner_query)
# conn.commit()

# # Create table if not exists
# create_table_query = '''CREATE TABLE IF NOT EXISTS laureanoengulian_coderhouse.big_five_weekly(
#                         "week_from" date not null,
#                         "open" decimal(38, 4) not null,
#                         "high" decimal(38, 4) not null,
#                         "low" decimal(38, 4) not null,
#                         "close" decimal(38, 4) not null,
#                         "volume" bigint not null,
#                         "symbol" varchar(15) not null,
#                         "avg" decimal(38,4) not null,
#                         "pk" varchar(60));
#                      '''

# cursor.execute(create_table_query)
# conn.commit()